In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from torch.utils.data import DataLoader

In [3]:
from cbow.dataset import CBOWDataset
from cbow.args import args
from cbow.datacontainer import DataContainer
args

Namespace(batch_size=128, checkpointer_name='cbow_classifier', checkpointer_prefix='cbow', device='cuda:3', early_stopping_criteria=5, learning_rate=0.0001, model_dir=PosixPath('../data/books/work_dir/models'), num_epochs=100, path=PosixPath('../data/books'), proc_dataset_csv=PosixPath('../data/books/frankenstein_with_splits.csv'), save_every=2, save_total=5, vectorizer_file=PosixPath('../data/books/work_dir/vectorizer.json'), work_dir=PosixPath('../data/books/work_dir'))

In [5]:
df = pd.read_csv(args.proc_dataset_csv)
dc = DataContainer(df, args.vectorizer_file, args.batch_size, is_load=True)

In [9]:
train_dl, val_dl, test_dl = dc.get_loaders()